# Group 07 - Code - Rotten Tomatoes Web Scraping
## Vedenikova_Vitalia_StudentB

In [ ]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import pickle
import sys
import csv
import warnings
from scipy import stats
from ast import literal_eval
pd.set_option('display.max_colwidth', None) #allows to see all the columns when working with data frames.

## 7. Enriching the Data 

In [ ]:
df = pd.read_csv('Vedenikova_Vitalia_studentB_stage3.csv', low_memory=False)

### 7.1 Identifying Outliers in terms of quantities of reviews

In [ ]:
review_counts = df["Movie_Title"].value_counts().sort_index()
review_counts = review_counts.to_frame()
review_counts.reset_index(inplace=True)

In [ ]:
review_counts.describe()

In [ ]:
Q1 = review_counts['count'].quantile(0.25)
Q3 = review_counts['count'].quantile(0.75)
IQR = Q3 - Q1
print(f'This is the lower boundary : {Q1-1.5*IQR}\nThis is the upper boundary: {Q3+1.5*IQR}')

In [ ]:
upper_boundary = review_counts['count'].mean() + 2*review_counts['count'].std()

In [ ]:
review_counts['count'] = review_counts['count'] > upper_boundary

In [ ]:
df = df.merge(review_counts,how='left', on='Movie_Title')
df.rename(columns={'count': 'Outlier_Reviews_Amount_(True/False)'}, inplace=True)

### 7.2 Percentage of difference between the critics' overall score and the audience's overall score 

In [ ]:
pd.options.mode.chained_assignment = None 
df['Difference_Critics_Audience'] = df['Tomatometer'] - df['Audience_Score']

### 7.3 Movies' box office revenue adjusted for inflation 

In [ ]:
df_inflation = pd.read_csv('../Code/Data/CPI_data.csv')
df_inflation.rename(columns={'CUUS0000SA0': 'CPI'}, inplace=True)

In [ ]:
df_inflation['DATE'].str.split('/')
df_inflation['DATE'] = df_inflation['DATE'].apply(lambda x : x[0:4])
df_inflation.drop_duplicates(subset='DATE', inplace=True)

In [ ]:
df['DATE'] = df['Release_Date_(Theaters)'].str[0:4]

In [ ]:
df_inflation['Multiplier'] = df_inflation['CPI'].iloc[-1] / df_inflation['CPI']
df_inflation.loc[29] = [int(2024), 302.408, 1.000000]

In [ ]:
df = pd.merge(df, df_inflation, how='left', on='DATE')

In [ ]:
df['Box_Office_Adjusted_Inflation_(USD)'] = df['Box_Office_(Gross_USA)']*df['Multiplier']

In [ ]:
df.fillna(0, inplace=True)
df.drop(['Multiplier','CPI','DATE'], axis=1, inplace=True)

df = df[['Movie_Title', 'Release_Date_(Theaters)','Rottentomatoes_URLs', 'Rating', 'Genre', 'Original_Language', 'Director', 
             'Producer','Writer', 'Cast_And_Crew', 'Distributor','Production_Co', 'Synopsis','Runtime', 'Box_Office_(Gross_USA)', 
             'Box_Office_Adjusted_Inflation_(USD)','Audience_Score', 'Tomatometer_State', 'Tomatometer', 'Critics_Concensus',
            'Difference_Critics_Audience','Review_Date','Author', 'Journal','Positive_Negative','Reviews',
             'Outlier_Reviews_Amount_(True/False)']]

In [ ]:
warnings.filterwarnings("ignore")
df['Box_Office_Adjusted_Inflation_(USD)'] = df['Box_Office_Adjusted_Inflation_(USD)'].fillna(0).astype(int)
df['Box_Office_Adjusted_Inflation_(USD)'].replace(0,np.nan, inplace= True)

In [ ]:
df.to_csv('Vedenikova_Vitalia_studentB_stage3.csv', sep=',',index=False, encoding='utf-8-sig')